# Pré-processamento
1. Texto em minúsculo
2. Remoção de URLs
3. Remoção de menções
3. Tokenização
4. Emoji Replace
5. Remoção de stopwords
6. Tratamento de Hashtags

## Separação em duas bases processadas: Com hashtags e Sem Hashtags

In [1]:
import pandas as pd
import numpy as np

import re
import pickle

from nltk.tokenize import TweetTokenizer

from collections import Counter

In [2]:
df1 = pd.read_csv('datasets\\base_original.csv')
classification = pd.read_csv('coleta\\users_cap.csv')

In [6]:
df1.drop(['username', 'to', 'permalink'], axis=1)  # print df hiding usernames and links

,date,replies,retweets,favorites,text,geo,mentions,hashtags,id
0,2019-01-31 23:59:17,0,0,0,☝️E jornalista sabichão apoiando reforma fasci...,NaN,NaN,NaN,1091123788236886017
1,2019-01-31 23:59:03,0,0,0,Isso é só uma demostração do que virar quando ...,NaN,NaN,NaN,1091123728140902400
2,2019-01-31 23:57:31,0,0,0,Governo diz que espera aprovar reforma da Prev...,NaN,NaN,NaN,1091123342583779328
3,2019-01-31 23:56:44,0,0,0,"Joice, mudadando de assunto, é verdade q o gov...",NaN,NaN,NaN,1091123144109289472
4,2019-01-31 23:56:20,1,0,2,meu pai foi muito bolsominion arrepedindo fala...,NaN,NaN,NaN,1091123043919892480
...,...,...,...,...,...,...,...,...,...
980572,2019-11-01 00:03:11,0,0,0,Amoedo fosse presidente estaria jantando com R...,NaN,NaN,NaN,1190056657331593218
980573,2019-11-01 00:02:37,0,0,2,Essa devia ter sido a primeira reforma. Depois...,NaN,NaN,NaN,1190056511407611904
980574,2019-11-01 00:00:52,0,0,1,Se for preciso podemos fazer uma vakinha p ela...,NaN,@iG,NaN,1190056074201698309
980575,2019-11-01 00:00:04,0,0,0,Sindicato tenta barrar reforma da previdência ...,NaN,NaN,NaN,1190055869079281664


In [7]:
# Get month number
months = []
for date in df1['date']:
    # spliting the date and getting the month
    month = date.split('-')[1]

    if month == '01':
        months.append('Janeiro')
    elif month == '02':
        months.append('Fevereiro')
    elif month == '03':
        months.append('Março')
    elif month == '04':
        months.append('Abril')
    elif month == '05':
        months.append('Maio')
    elif month == '06':
        months.append('Junho')
    elif month == '07':
        months.append('Julho')
    elif month == '08':
        months.append('Agosto')
    elif month == '09':
        months.append('Setembro')
    elif month == '10':
        months.append('Outubro')
    else:
        months.append('Novembro')

In [8]:
# Create a new dataframe with columns of interest
df2 = df1[['username', 'text']]
df2.insert(0, 'month', months)

In [9]:
df2.drop('username', axis=1)  # print hiding usernames

,month,text
0,Janeiro,☝️E jornalista sabichão apoiando reforma fasci...
1,Janeiro,Isso é só uma demostração do que virar quando ...
2,Janeiro,Governo diz que espera aprovar reforma da Prev...
3,Janeiro,"Joice, mudadando de assunto, é verdade q o gov..."
4,Janeiro,meu pai foi muito bolsominion arrepedindo fala...
...,...,...
980572,Novembro,Amoedo fosse presidente estaria jantando com R...
980573,Novembro,Essa devia ter sido a primeira reforma. Depois...
980574,Novembro,Se for preciso podemos fazer uma vakinha p ela...
980575,Novembro,Sindicato tenta barrar reforma da previdência ...


In [10]:
cap_dict = {}  # key: username     value: cap value
for i in classification.itertuples():
    # i=0(index)   i=1(username)   i=2(display_score)  i=3(cap)
    cap_dict.update({i[1].lower() : i[3]})

In [11]:
cap = []  
for i in df1['username']:
    # handling error (user:'85229524' -> cap = NaN)
    if i != '85229524':
        cap.append(cap_dict[i.lower()])
    else:
        cap.append(np.nan)

df2.insert(2, 'cap', cap)

In [13]:
df2.drop('username', axis=1)

,month,cap,text
0,Janeiro,0.660850,☝️E jornalista sabichão apoiando reforma fasci...
1,Janeiro,NaN,Isso é só uma demostração do que virar quando ...
2,Janeiro,0.866291,Governo diz que espera aprovar reforma da Prev...
3,Janeiro,0.260368,"Joice, mudadando de assunto, é verdade q o gov..."
4,Janeiro,NaN,meu pai foi muito bolsominion arrepedindo fala...
...,...,...,...
980572,Novembro,0.743721,Amoedo fosse presidente estaria jantando com R...
980573,Novembro,0.346921,Essa devia ter sido a primeira reforma. Depois...
980574,Novembro,0.376790,Se for preciso podemos fazer uma vakinha p ela...
980575,Novembro,0.847463,Sindicato tenta barrar reforma da previdência ...


In [14]:
def open_pickle(filename):
    """Function to open a pickle file and store into a variable"""
    
    with open(filename, 'rb') as pickle_file:
        new_data = pickle.load(pickle_file)
        return new_data

In [15]:
def lower_data(data):
    lst = []  # list with lower data

    for text in data:
        lst.append(text.lower())
    
    return lst

In [16]:
def remove_URL(data):
    """Remove all URLs and store the new texts into a list"""

    lst = []  # list of texts without links

    for text in data:
        # new_text = re.sub(r'https?://\S+', '', text)
        new_text = re.sub(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', '', text)
        new_text = re.sub(r'www?.(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', '', new_text)
        lst.append(new_text)
    return lst

In [17]:
def remove_mentions(data):
    lst = []

    for text in data:
        new_text = re.sub(r'@[\w]*', '', text)
        lst.append(new_text)
    return lst

In [18]:
def remove_hashtags(data):
    lst = []

    for text in data:
        new_text = re.sub(r'#[\w]*', '', text)
        lst.append(new_text)
    return lst

In [19]:
def tokenize(data):
    
    lst = []  # list of tokens

    tokenizer = TweetTokenizer()
    
    for text in data:
        lst.append(tokenizer.tokenize(text))
    
    return lst

In [20]:
def untokenize(data):
    
    lst = []  # list of phrases
    
    for tokens in data:
        new_phrase = []  # list containing all tokens from phrase
        
        for token in tokens:
            new_phrase.append(token)
        
        # transforming the list into a phrase
        lst.append(' '.join(new_phrase))
        
    return lst

In [21]:
def replace_emoticons(data, emoticon_dict):
    """Function to replace emojis with "positive", "negative" and "neutral" 
    data -> list of tokens
    emoji_list -> list of classified emoticons
    """

    lst = []  # result list containing token lists

    for token_list in data:
        
        new_tokens = []
        
        for token in token_list:
            if token in emoticon_dict['emoticon_positivo']:
                token = 'emoticon_positivo'

            elif token in emoticon_dict['emoticon_negativo']:
                token = 'emoticon_negativo'

            elif token in emoticon_dict['emoticon_neutro']:
                token = 'emoticon_neutro'
                
            new_tokens.append(token)
        
        # Append new token list in result list            
        lst.append(new_tokens)

    return lst

In [22]:
def standardize_tokens(data, abbreviation_dict):
    """Function to replace abbreviations for complete words in tweets"""
    
    lst = []  # tweets in form of a list of tokens without abbreviations
    
    for token_list in data:
        
        new_tokens = []
        
        for token in token_list:
            if token.lower() in abbreviation_dict:
                new_tokens.append(abbreviation_dict[token.lower()])
            else:
                new_tokens.append(token)
                
        lst.append(new_tokens)
        
    return lst

In [23]:
def remove_stopwords(data, stopwords):
    """Function to remove stopwords from tokens
    data -> list containing lists of tokens
    stopwords -> stopwords who will be removed from tokens
    """ 
    
    lst = []  # new tokens without stopwords

    for token_list in data:

        new_tokens = []

        for token in token_list:
            if token.lower() in stopwords:
                continue
            else:
                new_tokens.append(token.lower())
        
        lst.append(new_tokens)
    
    return lst

## Dicionários (Abreviações e Emoticons) e Stopwords
Referência: https://github.com/RafaelDRicci/PythonSentimentAnalysis/tree/main/listas_dicionarios

In [24]:
emoji_dict = open_pickle('files\\emoticon_list.pkl')
abbreviation_dict = open_pickle('files\\abreviacoes.pkl')
stopwords = open_pickle('files\\stopwords_customizadas.pkl')

In [25]:
lower_text = lower_data(df2['text'])

# adding a column to pre-processed text
df2.insert(4, 'processed_text', lower_text)

In [26]:
# Removing URLs and mentions
texts = remove_URL(df2['processed_text'])
texts = remove_mentions(texts)
#texts = remove_hashtags(texts)

# Tokenizing
tokens = tokenize(texts)

# Replacing abbreviations and emojis
tokens = standardize_tokens(tokens, abbreviation_dict)
tokens = replace_emoticons(tokens, emoji_dict)

# Removing Stopwords and Untokenizing
tokens = remove_stopwords(tokens, stopwords)
texts = untokenize(tokens)

# Updating 'processed_text' column
df2 = df2.copy()
new_values = pd.Series(texts, name='processed_text')
df2.update(new_values)

In [27]:
df2.drop('username', axis=1)

,month,cap,text,processed_text
0,Janeiro,0.660850,☝️E jornalista sabichão apoiando reforma fasci...,emoticon_positivo ️ jornalista sabichão apoian...
1,Janeiro,NaN,Isso é só uma demostração do que virar quando ...,demostração virar chegar congresso
2,Janeiro,0.866291,Governo diz que espera aprovar reforma da Prev...,governo diz espera aprovar 1º semestre
3,Janeiro,0.260368,"Joice, mudadando de assunto, é verdade q o gov...",joice mudadando assunto verdade governo pedind...
4,Janeiro,NaN,meu pai foi muito bolsominion arrepedindo fala...,pai bolsominion arrepedindo falando discurso m...
...,...,...,...,...
980572,Novembro,0.743721,Amoedo fosse presidente estaria jantando com R...,amoedo presidente estaria jantando rodrigo mai...
980573,Novembro,0.346921,Essa devia ter sido a primeira reforma. Depois...,devia ter sido primeira modernização decente c...
980574,Novembro,0.376790,Se for preciso podemos fazer uma vakinha p ela...,preciso podemos fazer vakinha ir pessoalmente ...
980575,Novembro,0.847463,Sindicato tenta barrar reforma da previdência ...,sindicato tenta barrar governo goiás justiça


In [28]:
def get_hashtags(data):
    """Function to return a dict(Counter) of hashtags"""
    hashtags = []

    tokens = tokenize(data)

    for token_list in tokens:

        for word in token_list:
            if word[0] == '#':
                hashtags.append(word)
    
    c1 = Counter(hashtags)

    return c1

In [31]:
def replace_accents(data):
    """Receives one list of strings and remove all accents from the strings in the list"""

    # Replacements
    rep_a = ['á', 'à', 'ã', 'â', 'ä']
    rep_e = ['é', 'ê', 'ë']
    rep_i = ['í', 'ï']
    rep_o = ['ó', 'õ', 'ô', 'ö']
    rep_u = ['ú', 'ü']

    lst = []
    
    for word in data:
        for letter in rep_a:
            word = word.replace(letter, 'a')

        for letter in rep_e:
            word = word.replace(letter, 'e')

        for letter in rep_i:
            word = word.replace(letter, 'i')

        for letter in rep_o:
            word = word.replace(letter, 'o')

        for letter in rep_u:
            word = word.replace(letter, 'u')
        
        lst.append(word)
    
    return lst

In [32]:
def normalize_hashtags(data):
    """
    Receives a list of tweets (data)
    Generates a dict of hashtags (keys) and hashtags without accents (values)
    Replace hashtags for the new ones using the generated dict
    """

    hashtags = []

    tokens = tokenize(data)

    for token_list in tokens:
        
        for word in token_list:
            if word[0] == '#':
                hashtags.append(word)
    
    # remove accents_from hashtags
    hashtags2 = replace_accents(hashtags)

    # Create a dict
    hs_replacement = {}
    
    for i in range(len(hashtags)):
        # key: Hashtag with accents  value = hashtag without accents
        key = hashtags[i]
        value = hashtags2[i]
        
        hs_replacement[key] = value
    
    lst = []

    # Replace hashtags (for hashtags without accents)
    for token_list in tokens:

        new_tokens = []

        for token in token_list:

            # verify if token is a hashtag
            if token in hs_replacement:
                new_tokens.append(hs_replacement[token])  # replace for hashtag without accents
            else:
                new_tokens.append(token)
        
        lst.append(new_tokens)
    
    # untokenize
    texts = untokenize(lst)
        
    return texts

In [33]:
texts = normalize_hashtags(df2['processed_text'])

In [34]:
hashtags = get_hashtags(texts)

In [35]:
hashtags.most_common(150)

[('#reformadaprevidencia', 53752),
 ('#previdencia', 4918),
 ('#reformadaprevidenciaja', 4771),
 ('#euapoionovaprevidencia', 4112),
 ('#brasilnasruas', 3039),
 ('#pacoteanticrime', 3030),
 ('#novaprevidencia', 3019),
 ('#reformadaprevidencianao', 2779),
 ('#dia26euvou', 2762),
 ('#bolsonaronossopresidente', 2494),
 ('#bolsonaro', 2419),
 ('#lutepelasuaaposentadoria', 2373),
 ('#brasil', 2361),
 ('#reformadeumtrilhao', 2340),
 ('#politica', 2302),
 ('#dia26brasilnasruas', 2283),
 ('#reformanao', 2135),
 ('#senadoaproveareforma', 2108),
 ('#naoareformadaprevidencia', 2097),
 ('#grevegeral', 1946),
 ('#reajaousuaprevidenciaacaba', 1854),
 ('#reformainjusta', 1761),
 ('#previdenciaoumorte', 1745),
 ('#reforma', 1579),
 ('#lulanacadeia', 1576),
 ('#euapoiosergiomoro', 1521),
 ('#globolixo', 1513),
 ('#lulalivre', 1473),
 ('#pacoteanticrimeja', 1472),
 ('#1trilhaosemprivilegios', 1424),
 ('#jornaldacultura', 1423),
 ('#previdenciadoguedes', 1420),
 ('#euapoioalavajato', 1393),
 ('#reformadap

In [36]:
# unique hashtags
print(f'Hashtags Únicas: {len(hashtags)}')

Hashtags Únicas: 27109


In [37]:
sum(hashtags.values())

343826

## Dicionário de Hashtags
Considerando as 150 hashtags mais utilizadas na base, foi feita uma classificação manual das hashtags mais relevantes em: `hashtag_positiva` e `hashtag_negativa`.<br>
As hashtags "neutras" não foram inseridas nesse dicionário, pois pretende-se verificar a mudança nos resultados ao comparar a classificação do modelo na base sem hashtags e na base com hashtags.<br>
Neste caso, não há necessidade de adicionar as hashtags "neutras" na análise, pois a ideia é verificar se possíveis tweets que antes recebiam a classificação neutra, agora passam a receber classificação positiva ou negativa, somente
adicionando ao modelo as hashtags mais relevantes (positivas e negativas).

In [38]:
hashtag_positiva = ['#reformadaprevidenciaja', '#euapoionovaprevidencia', '#dia26euvou', '#dia26brasilnasruas', 
'#senadoaproveareforma', '#previdenciaoumorte', '#reformadaprevidenciasim', '#dia30euvou','oureformaouquebra',
'#votesimpelareforma', '#dia26nasruas', '#brasilexigereformaccj', '#dia30pelobrasil', '#novaprevidenciaja',
'#euapoioanovaprevidencia', '#senadoaproveaprevidencia', '#previdenciaanticrimeja', '#brasilquerreforma',
'#previdencianaopodeesperar', '#reformaja', '#previdenciaaprovaja','#novaprevidenciasim', '#brasiltempressa','#reformasja',
'#dia26reformasja', '#acelerareformadaprevidencia','#reformasim','#previdenciasim', '#dia30vemprarua', '#reformavotesim']

hashtag_negativa = ['#reformadaprevidencianao', '#lutepelasuaaposentadoria', '#reformanao', '#naoareformadaprevidencia',
'#grevegeral', '#reajaousuaprevidenciaacaba', '#reformainjusta', '#quereformaeessa', '#grevepelobrasil', '#grevegeral14j',
'#greve14j', '#14j', '#reformacomprada', '#tsunamidaeducaçao', '#brasilbarrareforma', '#areformateengana', '#sextatemgreve',
'#pressaopelaaposentadoria', '#reformapenacova', '#aposentadoriasimarmasnao', '#paremareformadaprevidencia', '#dia13erua']

hashtag_dict = {}

hashtag_dict['hashtag_positiva'] = hashtag_positiva
hashtag_dict['hashtag_negativa'] = hashtag_negativa

print(hashtag_dict)

{'hashtag_positiva': ['#reformadaprevidenciaja', '#euapoionovaprevidencia', '#dia26euvou', '#dia26brasilnasruas', '#senadoaproveareforma', '#previdenciaoumorte', '#reformadaprevidenciasim', '#dia30euvou', 'oureformaouquebra', '#votesimpelareforma', '#dia26nasruas', '#brasilexigereformaccj', '#dia30pelobrasil', '#novaprevidenciaja', '#euapoioanovaprevidencia', '#senadoaproveaprevidencia', '#previdenciaanticrimeja', '#brasilquerreforma', '#previdencianaopodeesperar', '#reformaja', '#previdenciaaprovaja', '#novaprevidenciasim', '#brasiltempressa', '#reformasja', '#dia26reformasja', '#acelerareformadaprevidencia', '#reformasim', '#previdenciasim', '#dia30vemprarua', '#reformavotesim'], 'hashtag_negativa': ['#reformadaprevidencianao', '#lutepelasuaaposentadoria', '#reformanao', '#naoareformadaprevidencia', '#grevegeral', '#reajaousuaprevidenciaacaba', '#reformainjusta', '#quereformaeessa', '#grevepelobrasil', '#grevegeral14j', '#greve14j', '#14j', '#reformacomprada', '#tsunamidaeducaçao', '

In [39]:
# storing hashtag dict in a pickle file
pickle_file = open('files\\hashtags_dict.pkl', 'wb')
pickle.dump(hashtag_dict, pickle_file)
pickle_file.close()

In [40]:
def replace_hashtags(data, hashtag_dict):
    """Function to replace hashtags with "positive" and "negative" 
    data -> data to replace hashtags
    emoji_list -> list of classified hashtags
    """

    lst = []

    tokens = tokenize(data)

    for token_list in tokens:
        
        new_tokens = []
        
        for token in token_list:
            if token in hashtag_dict['hashtag_positiva']:
                token = 'hashtag_positiva'

            elif token in hashtag_dict['hashtag_negativa']:
                token = 'hashtag_negativa'
                
            new_tokens.append(token)
        
        # Append new token list in result list            
        lst.append(new_tokens)
    
    # untokenize
    texts = untokenize(lst)

    return texts

In [41]:
new_texts = replace_hashtags(texts, hashtag_dict)

In [48]:
# Updating 'processed_text' column
df2 = df2.copy()
new_values = pd.Series(new_texts, name='processed_text')
df2.update(new_values)

In [49]:
df2.drop('username', axis=1)

,month,cap,text,processed_text
0,Janeiro,0.660850,☝️E jornalista sabichão apoiando reforma fasci...,emoticon_positivo ️ jornalista sabichão apoian...
1,Janeiro,NaN,Isso é só uma demostração do que virar quando ...,demostração virar chegar congresso
2,Janeiro,0.866291,Governo diz que espera aprovar reforma da Prev...,governo diz espera aprovar 1º semestre
3,Janeiro,0.260368,"Joice, mudadando de assunto, é verdade q o gov...",joice mudadando assunto verdade governo pedind...
4,Janeiro,NaN,meu pai foi muito bolsominion arrepedindo fala...,pai bolsominion arrepedindo falando discurso m...
...,...,...,...,...
980572,Novembro,0.743721,Amoedo fosse presidente estaria jantando com R...,amoedo presidente estaria jantando rodrigo mai...
980573,Novembro,0.346921,Essa devia ter sido a primeira reforma. Depois...,devia ter sido primeira modernização decente c...
980574,Novembro,0.376790,Se for preciso podemos fazer uma vakinha p ela...,preciso podemos fazer vakinha ir pessoalmente ...
980575,Novembro,0.847463,Sindicato tenta barrar reforma da previdência ...,sindicato tenta barrar governo goiás justiça


In [50]:
# df2.insert(3, 'to', df1['to'])
df2.insert(3, 'retweets', df1['retweets'])
# df2.insert(4, 'mentions', df1['mentions'])
df2.insert(4, 'favorites', df1['favorites'])

In [52]:
df2.drop('username', axis=1)

,month,cap,retweets,favorites,text,processed_text
0,Janeiro,0.660850,0,0,☝️E jornalista sabichão apoiando reforma fasci...,emoticon_positivo ️ jornalista sabichão apoian...
1,Janeiro,NaN,0,0,Isso é só uma demostração do que virar quando ...,demostração virar chegar congresso
2,Janeiro,0.866291,0,0,Governo diz que espera aprovar reforma da Prev...,governo diz espera aprovar 1º semestre
3,Janeiro,0.260368,0,0,"Joice, mudadando de assunto, é verdade q o gov...",joice mudadando assunto verdade governo pedind...
4,Janeiro,NaN,0,2,meu pai foi muito bolsominion arrepedindo fala...,pai bolsominion arrepedindo falando discurso m...
...,...,...,...,...,...,...
980572,Novembro,0.743721,0,0,Amoedo fosse presidente estaria jantando com R...,amoedo presidente estaria jantando rodrigo mai...
980573,Novembro,0.346921,0,2,Essa devia ter sido a primeira reforma. Depois...,devia ter sido primeira modernização decente c...
980574,Novembro,0.376790,0,1,Se for preciso podemos fazer uma vakinha p ela...,preciso podemos fazer vakinha ir pessoalmente ...
980575,Novembro,0.847463,0,0,Sindicato tenta barrar reforma da previdência ...,sindicato tenta barrar governo goiás justiça


In [54]:
df2.to_csv('base_processada.csv', index=False)